# Big Data - Dashboard
## ALBICHARI Kaïs - D'HOSE Tanguy - ULB

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
import sys
import os
from datetime import datetime
from pathlib import Path
from kafka import KafkaConsumer
from kafka.errors import KafkaError

In [3]:
import headtail
headtail.head('data/data.conv.txt', 5)

['2017-03-31 03:38:16.508 1-0 122.153 2.03397\n',
 '2017-03-31 03:38:15.967 1-1 -3.91901 2.09397\n',
 '2017-03-31 03:38:16.577 1-2 11.04 2.07397\n',
 '2017-02-28 00:59:16.359 1-0 19.9884 2.74964\n',
 '2017-02-28 00:59:16.803 1-1 37.0933 2.76964\n']

In [4]:
STREAM_IN = "stream-in"

sc = SparkContext("local[*]", "test")
sc.setLogLevel("WARN")   #Make sure warnings and errors observed by spark are printed.

ssc = StreamingContext(sc, 5)  #generate a mini-batch every 5 seconds

filestream = ssc.textFileStream(STREAM_IN) #monitor new files in folder stream-IN

In [6]:


def parseRow(row):
    '''parses a single row into a dictionary'''
    v = row.split(" ")
    try:
        return [{"topic": v[0],
                 "time": datetime.strptime(v[1] + " "+ v[2], "%Y-%m-%d %H:%M:%S.%f"),
                 "p-i": values[3],
                 "measurement": float(values[4]),
                 "voltage": float(values[5])}]
    except Exception as err:
        print("Unexpected error: %s" % (err))


def checkAttributes(sensor_type, space_tag, time_tag):
    if sensor_type < 0 or sensor_type > 3:
        print("Sensor type value is not supported. Give a value in [0,3]")
        return False
    elif space_tag < 0 or space_tag > 2:
        print("Space tag value is not supported. Give a value in [0,2].")
        print("0: grouped per space")
        print("1: grouped per municipality")
        print("2: grouped for the entirety of Brussels")
        return False
    elif time_tag <0 or time_tag > 4:
        print("Time tag value is not supported. Give a value in [0,4].")
        print("0: last 24h")
        print("1: last 2 days")
        print("2: last week")
        print("3: last month")
        print("4: last year")
        return False
    else:
        return True
        

def basicStats(sensor_type, space_tag, time_tag):
    """
    Basic statistics (min, max, avg) about sensor readings grouped according to different granularities
    in space and time.
    sensor_type: type of the sensor (0 to 3)
    space_tag: defines granularity in space (0 to 2)
    time_tag: defines granularity in time (0 to 4)
    """
    if not checkAttributes(sensor_type, space_tag, time_tag):
        exit()
    filestream = ssc.textFileStream(STREAM_IN) #monitor new files in folder stream-IN

    rows = filestream.flatMap(parseRow)




basicStats(1,2,0)